# Sentiment Analysis Techniques

In [1]:
import os
import sys


# This adds the 'src' directory to the Python path.
src_rel_path = "../src"
src_abs_path = os.path.abspath(src_rel_path)

if src_abs_path not in sys.path:
    sys.path.append(src_abs_path)
    print(f"Added '{src_abs_path}' to sys.path")
else:
    print(f"'{src_abs_path}' is already in sys.path")

import database as db
import pandas as pd

Added '/Users/shubxam/Code/Nifty-500-Live-Sentiment-Analysis/src' to sys.path


In [2]:
conn = db.DatabaseManager()
articles_df = conn.get_articles(has_sentiment=False, n=500)

In [3]:
headlines = articles_df['headline'].to_list()
headlines

['Dividend stocks: HDFC Bank, Tata Elxsi, ICICI Bank, Infosys, others to declare dividend next week',
 'Shobana Kamineni bets on digital growth, Apollo 24/7 to turn profitable by fourth quarter of FY26',
 'Q4 results 2025: HDFC Bank, Yes Bank, Infosys, Wipro, others to declare earnings THIS week | Full list here',
 'HDFC Bank Q4 Results 2025 Date and Time: India’s Largest Lender to Announce Earnings Next Week',
 'Infosys (INFY) Expected to Announce Quarterly Earnings on Thursday',
 "India's non-coking coal imports drop 8% y-o-y in FY'25. Will the trend sustain?",
 "Soft prices, rising input costs likely to make a dent in cement companies' profits",
 'Wipro (WIT) Projected to Post Earnings on Wednesday',
 'Stock Market News, April 11, 2025: Stocks Rise Despite Recession Fears; Treasury Yield Surges',
 'Vodafone Idea, Suzlon Energy, Kotak Mahindra Bank & Bajaj Finance: What Jay Thakkar of ICICI Sec says on these 4 stocks',
 'Insider trading erodes investor confidence: Sanjiv Mehta',
 "Ma

## External APIs

### HuggingFace Inference API


In [4]:
from transformers.models.bert import BertTokenizer, BertForSequenceClassification
from transformers.pipelines import pipeline

#### kdave/FineTuned_Finbert

In [5]:
finbert = BertForSequenceClassification.from_pretrained(
    'kdave/FineTuned_Finbert',
    num_labels=3,
    subfolder='finbert',  # Specify the subdirectory containing the model files
    use_safetensors=True  # Use safe tensors
)

tokenizer = BertTokenizer.from_pretrained(
    'kdave/FineTuned_Finbert',
    subfolder='finbert'  # Also need this for the tokenizer
)

In [6]:
# set top_k=1 to get the most likely label or top_k=None to get all labels
# device=-1 means CPU
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, device=-1, top_k=1, framework="pt")

Device set to use cpu


In [7]:
nlp_res = nlp(headlines, batch_size=512)

#### yiyanghkust/finbert-tone
Much Superior than kdave/FineTuned_Finbert

In [8]:
finbert_1 = BertForSequenceClassification.from_pretrained(
    'yiyanghkust/finbert-tone',
    num_labels=3,
    use_safetensors=True  # Use safe tensors
    
)

tokenizer_1 = BertTokenizer.from_pretrained(
    'yiyanghkust/finbert-tone'
)

In [9]:
# set top_k=1 to get the most likely label or top_k=None to get all labels
# device=-1 means CPU
nlp_1 = pipeline("sentiment-analysis", model=finbert_1, tokenizer=tokenizer_1, device=-1, top_k=1, framework="pt")

Device set to use cpu


In [18]:
nlp_1_res = nlp_1(headlines, batch_size=512)

#### comparison

In [28]:
flattened_data_0 = [item[0] for item in nlp_res]
sentiment_df_0 = pd.DataFrame(flattened_data_0)
flattened_data_1 = [item[0] for item in nlp_1_res]
sentiment_df_1 = pd.DataFrame(flattened_data_1)

sentiment_df = pd.merge(sentiment_df_0, sentiment_df_1, left_index=True, right_index=True, suffixes=('_finbert', '_finbert_1'))
sentiment_df['headline'] = headlines[:len(sentiment_df)]

In [31]:
sentiment_df.to_csv('finbert_sentiment.csv', index=False)

In [25]:
sentiment_df.rename(
    columns={
        'label': 'label_2',
        'score': 'score_2'
    },
    inplace=True
)